In [12]:
import os
from glob import glob
import numpy as np
import nibabel as nib
import nibabel.processing
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
from torchvision import transforms
import torch.nn as nn
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm
from operator import add
import matplotlib.pyplot as plt
import easydict
import dicom2nifti
import dicom2nifti.settings as settings
from scipy.ndimage import zoom
settings.disable_validate_slice_increment()


# mra--> nii file 변환작업

In [11]:
# dcm_path = '/home/chaeeun/MRI100_NECKTOFMRA/*'
# nii_path = '/home/chaeeun/Desktop/brain disease prediction/fileIO/data_niis'

In [12]:
# dcm_list_path = '/home/chaeeun/Necktofmra'
# dcm_list =  os.listdir(dcm_list_path)
# dcm_list.sort()

In [ ]:
# #dcm --> nii code
# #for i, patient in enumerate(glob(dcm_path)):nonononononono please don't use this line
# for f in tqdm(dcm_list):
#     dicom2nifti.dicom_series_to_nifti(dcm_list_path +'/'+ f, os.path.join(nii_path, f + '.nii.gz'))#nii_path에 파일 추가
#     print(f + 'done')#well done check

## 끝

# 

In [2]:
nii_path = '/home/chaeeun/Desktop/brain disease prediction/fileIO/data_niis/'
files = os.listdir(nii_path)
files.sort()
files = files[1:]#ipynb_checkpoints file name remove...
len(files), files[0]

(721, '10436442_20170329_095856_MR.nii.gz')

# resize (192, 252, 272 -> 232, z축)

In [16]:
def cut_to_232(files, origin_path, new_path):
    for f in tqdm(files):
        path = os.path.join(origin_path, f)
        img = nib.load(path)
        if img.shape[2] == 192:
            affine = img.affine
            img = img.get_fdata()
            zoom_img = zoom(img, (1, 1, 232/192))
            new_img = nib.Nifti1Image(zoom_img, affine = affine)
            nib.save(new_img, os.path.join(new_path, f))
        elif img.shape[2] == 232:
            nib.save(img, os.path.join(new_path, f))
            
        elif img.shape[2] == 233:
            sliced_img = img.slicer[:, :, 1:]
            nib.save(sliced_img, os.path.join(new_path, f))
            
        elif img.shape[2] == 252:
            sliced_img = img.slicer[:, :, 20:]
            nib.save(sliced_img, os.path.join(new_path, f))
            
        elif img.shape[2] == 272:
            sliced_img = img.slicer[:, :, 40:]
            nib.save(sliced_img, os.path.join(new_path, f))



In [17]:
origin_path = '/home/chaeeun/Desktop/brain disease prediction/fileIO/data_niis/'
new_path = '/home/chaeeun/Desktop/brain disease prediction/fileIO/data_nii_232/'


In [ ]:
cut_to_232(files, origin_path, new_path)

 22%|██▏       | 159/721 [07:23<23:32,  2.51s/it]

### resize done

In [20]:
#checking there is only 232
s  = set()
for i in files:
    path = os.path.join(new_path, i)
    img = nib.load(path)
    s.add(img.shape[2])
print(s)


{232}


# slicing start _slice data 보류_

In [30]:
def slice_data(files, new_path, a, b, c, d, e, f):
    data = []
    for f in tqdm(files):
        path = os.path.join(new_path, f)
        img = nib.load(path)
        a = int(a)
        b = int(b)
        c = int(c)
        d = int(d)
        e = int(e)
        f = int(f)
        sliced_img = img.slicer[a:b, c:d, e:f]
        img = sliced_img.get_fdata()
        img = np.asarray(np.float32(img))
        img = img.reshape(1, b-a, d-c, f-e)
        data.append(img)
    return data

In [21]:
data = []
for f in tqdm(files):
    path = os.path.join(new_path, f)
    img = nib.load(path)
    sliced_img = img.slicer[190:330, 220:350, 70:232]
    img = sliced_img.get_fdata()
    img = np.asarray(np.float32(img))
    img = img.reshape(1, 140, 130, 162)
    data.append(img)

100%|██████████| 721/721 [12:14<00:00,  1.02s/it]


In [22]:
type(data)
data = np.array(data,dtype='float32') 

In [23]:
data.shape

(721, 1, 140, 130, 162)

In [24]:
# np.save('./sliced_datas/sliced_435', data, allow_pickle=True, fix_imports=True)
# 0 : [190:330, 220:350, 70:220], 140 130 150
# np.save('./sliced_datas/sliced_757', data, allow_pickle=True, fix_imports=True)
# 1 : [180:350, 200:350, 60:232] 170 150 172메모리 아웃
np.save('./sliced_datas/sliced_436', data, allow_pickle=True, fix_imports=True)
# 0 : [190:330, 220:350, 70:232] 140 130 162
# np.save('./sliced_datas/sliced_746', data, allow_pickle=True, fix_imports=True)
# 0 : [170:340, 180:320, 70:232] 170 140 162
# np.save('./sliced_datas/sliced_747', data, allow_pickle=True, fix_imports=True)
# 0 : [170:340, 180:320, 60:232] 170 140 172




In [31]:
data0 = slice_data(files, new_path, 190, 330, 220, 350, 70, 220)
np.save('./sliced_datas/sliced_435', data0, allow_pickle=True, fix_imports=True)
data3 = slice_data(files, new_path, 170, 340, 180, 320, 70, 232)
np.save('./sliced_datas/sliced_746', data3, allow_pickle=True, fix_imports=True)
data4 = slice_data(files, new_path, 170, 340, 180, 320, 60, 232)
np.save('./sliced_datas/sliced_747', data4, allow_pickle=True, fix_imports=True)





  0%|          | 0/721 [00:00<?, ?it/s]


ValueError: invalid literal for int() with base 10: '10436442_20170329_095856_MR.nii.gz'